# **London Underground ETL**

## Objectives

* Clean and merge Footfall and Station Coordinates datasets

## Inputs

* To run this notebook the StationFootfall and Stations datasets are required 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [19]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\jackr\\OneDrive\\Desktop\\my_projects\\Project-1'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [20]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [21]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\jackr\\OneDrive\\Desktop\\my_projects'

# Section 1

* Retrieving datasets and checking values

Importing necessary libraries

In [22]:
import pandas as pd

Parsing CSV files into DataFrames

In [23]:
Footfall = pd.read_csv(r'c:\Users\jackr\OneDrive\Desktop\my_projects\Project-1\Dataset\Dirty\StationFootfall_2024_2025 .csv')
Station_Coords = pd.read_csv(r'c:\Users\jackr\OneDrive\Desktop\my_projects\Project-1\Dataset\Dirty\Stations_20180921.csv')

Checking variable types 

In [24]:
Footfall.info()
Station_Coords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249121 entries, 0 to 249120
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   TravelDate     249121 non-null  int64 
 1   DayOfWeek      249121 non-null  object
 2   Station        249121 non-null  object
 3   EntryTapCount  249121 non-null  int64 
 4   ExitTapCount   249121 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 9.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FID       479 non-null    int64  
 1   OBJECTID  479 non-null    int64  
 2   NAME      479 non-null    object 
 3   EASTING   479 non-null    int64  
 4   NORTHING  479 non-null    int64  
 5   LINES     271 non-null    object 
 6   NETWORK   479 non-null    object 
 7   Zone      479 non-null    int64  
 8   x         479 non-null    float6

^ We can see that the 'x' and 'y' values are floats, this is ideal and important in creating map scatter plots later on

Displaying head of each DataFrame

In [25]:
Footfall.head()

,TravelDate,DayOfWeek,Station,EntryTapCount,ExitTapCount
0,20240101,Monday,Abbey Road DLR,395,375
1,20240101,Monday,Abbey Wood,5898,5963
2,20240101,Monday,Acton Central,609,474
3,20240101,Monday,Acton Main Line,1717,1710
4,20240101,Monday,Acton Town,2928,3334


In [33]:
Station_Coords.head()

,FID,OBJECTID,NAME,EASTING,NORTHING,LINES,NETWORK,Zone,x,y
0,0,78,Temple,530959,180803,"District, Circle",London Underground,1,-0.112644,51.510474
1,1,79,Blackfriars,531694,180893,"District, Circle",London Underground,1,-0.102020,51.511114
2,2,80,Mansion House,532354,180932,"District, Circle",London Underground,1,-0.092495,51.511306
3,3,81,Cannon Street,532611,180900,"District, Circle",London Underground,1,-0.088801,51.510963
4,4,82,Monument,532912,180824,"District, Circle",London Underground,1,-0.084502,51.510209


---

# Section 2

* Merging and Cleaning

Filtering non-underground stations

In [37]:
Station_Coords = Station_Coords.query("NETWORK == 'London Underground'")

# Checking Networks have been filtered correctly          
Station_Coords.query("NETWORK == ['London Overground', 'Tramlink', 'DLR', 'TfL Rail']")
                            

,FID,OBJECTID,NAME,EASTING,NORTHING,LINES,NETWORK,Zone,x,y


^ No rows appear when querying our filtered stations, they are no longer in the DataFrame

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)


IndentationError: expected an indented block after 'try' statement on line 2 (553063055.py, line 5)